# GraphStore: Neo4j Database Basic Examples

This notebook demonstrates how to use the `Neo4jGraphStore` in `dapr-agents` for basic graph-based tasks. We will explore:

* Initializing the `Neo4jGraphStore` class.
* Adding sample nodes.
* Adding one sample relationship.
* Querying graph database.
* Resseting database.

## Install Required Libraries

Ensure dapr_agents and neo4j are installed:

In [ ]:
!pip install dapr-agents python-dotenv neo4j

## Import Environment Variables

Load your API keys or other configuration values using `dotenv`.

In [1]:
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from a `.env` file

True

## Enable Logging

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

## Deploy Neo4j Graph Database as Docker Container

In [ ]:
#docker run \
#--restart always \
#--publish=7474:7474 --publish=7687:7687 \
#--env NEO4J_AUTH=neo4j/graphwardog \
#--volume=neo4j-data \
#--name neo4j-apoc \
#--env NEO4J_apoc_export_file_enabled=true \
#--env NEO4J_apoc_import_file_enabled=true \
#--env NEO4J_apoc_import_file_use__neo4j__config=true \
#--env NEO4J_PLUGINS=\[\"apoc\"\] \
#neo4j:latest

## Initialize Neo4jGraphStore

Set the `NEO4J_URI`, `NEO4J_USERNAME` and `NEO4J_PASSWORD` variables in a `.env` file. The URI can be set to `bolt://localhost:7687`.

In [3]:
from dapr_agents.storage.graphstores.neo4j import Neo4jGraphStore
import os

# Initialize Neo4jGraphStore
graph_store = Neo4jGraphStore(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    database="neo4j"
)

INFO:dapr_agents.storage.graphstores.neo4j.client:Successfully created the driver for URI: bolt://localhost:7687
INFO:dapr_agents.storage.graphstores.neo4j.base:Neo4jGraphStore initialized with database neo4j


In [4]:
# Test the connection
assert graph_store.client.test_connection(), "Connection to Neo4j failed"
print("Neo4j connection successful")

INFO:dapr_agents.storage.graphstores.neo4j.client:Connected to Neo4j Kernel version 5.15.0 (community edition)


Neo4j connection successful


## Add Sample Nodes
Create and add nodes to the graph store:

In [5]:
from dapr_agents.types import Node

# Sample nodes
nodes = [
    Node(
        id="1",
        label="Person",
        properties={"name": "Alice", "age": 30},
        additional_labels=["Employee"]
    ),
    Node(
        id="2",
        label="Person",
        properties={"name": "Bob", "age": 25},
        additional_labels=["Contractor"]
    )
]

# Add nodes
graph_store.add_nodes(nodes)
print("Nodes added successfully")


INFO:dapr_agents.storage.graphstores.neo4j.base:Processed batch 1/1
INFO:dapr_agents.storage.graphstores.neo4j.base:Nodes with label `Person` added successfully.


Nodes added successfully


## Add Sample Relationship
Create and add a relationship to the graph store:

In [6]:
from dapr_agents.types import Relationship

# Sample relationships
relationships = [
    Relationship(
        source_node_id="1",
        target_node_id="2",
        type="KNOWS",
        properties={"since": "2023"}
    )
]

# Add relationships
graph_store.add_relationships(relationships)
print("Relationships added successfully")

INFO:dapr_agents.storage.graphstores.neo4j.base:Processed batch 1/1
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (b))} {position: line: 3, column: 25, offset: 45} for query: '\n            UNWIND $data AS rel\n            MATCH (a {id: rel.source_node_id}), (b {id: rel.target_node_id})\n            MERGE (a)-[r:`KNOWS`]->(b)\n            ON CREATE SET r.creat

Relationships added successfully


## Query Graph

In [7]:
query = "MATCH (n) RETURN n"
results = graph_store.query(query)
print("Nodes in the database:")
for record in results:
    print(record)

INFO:dapr_agents.storage.graphstores.neo4j.base:Query executed successfully: MATCH (n) RETURN n | Time: 0.06 seconds | Results: 2


Nodes in the database:
{'n': {'createdAt': '2025-03-04T10:55:57.109885Z', 'name': 'Alice', 'id': '1', 'age': 30, 'updatedAt': '2025-03-04T10:55:57.109885Z'}}
{'n': {'createdAt': '2025-03-04T10:55:57.109885Z', 'name': 'Bob', 'id': '2', 'age': 25, 'updatedAt': '2025-03-04T10:55:57.109885Z'}}


In [8]:
query = """
MATCH (a)-[r]->(b)
RETURN a.id AS source, b.id AS target, type(r) AS type, properties(r) AS properties
"""
results = graph_store.query(query)
print("Relationships in the database:")
for record in results:
    print(record)

INFO:dapr_agents.storage.graphstores.neo4j.base:Query executed successfully: 
MATCH (a)-[r]->(b)
RETURN a.id AS source, b.id AS target, type(r) AS type, properties(r) AS properties
 | Time: 0.07 seconds | Results: 1


Relationships in the database:
{'source': '1', 'target': '2', 'type': 'KNOWS', 'properties': {'updatedAt': '2025-03-04T10:55:59.835379Z', 'createdAt': '2025-03-04T10:55:59.835379Z', 'since': '2023'}}


In [9]:
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
"""
results = graph_store.query(query)
print("Nodes and relationships in the database:")
for record in results:
    print(record)

INFO:dapr_agents.storage.graphstores.neo4j.base:Query executed successfully: 
MATCH (n)-[r]->(m)
RETURN n, r, m
 | Time: 0.05 seconds | Results: 1


Nodes and relationships in the database:
{'n': {'createdAt': '2025-03-04T10:55:57.109885Z', 'name': 'Alice', 'id': '1', 'age': 30, 'updatedAt': '2025-03-04T10:55:57.109885Z'}, 'r': ({'createdAt': '2025-03-04T10:55:57.109885Z', 'name': 'Alice', 'id': '1', 'age': 30, 'updatedAt': '2025-03-04T10:55:57.109885Z'}, 'KNOWS', {'createdAt': '2025-03-04T10:55:57.109885Z', 'name': 'Bob', 'id': '2', 'age': 25, 'updatedAt': '2025-03-04T10:55:57.109885Z'}), 'm': {'createdAt': '2025-03-04T10:55:57.109885Z', 'name': 'Bob', 'id': '2', 'age': 25, 'updatedAt': '2025-03-04T10:55:57.109885Z'}}


## Reset Graph

In [10]:
graph_store.reset()
print("Graph database has been reset.")

INFO:dapr_agents.storage.graphstores.neo4j.base:Database reset successfully


Graph database has been reset.


In [11]:
query = "MATCH (n) RETURN n"
results = graph_store.query(query)
print("Nodes in the database:")
for record in results:
    print(record)

INFO:dapr_agents.storage.graphstores.neo4j.base:Query executed successfully: MATCH (n) RETURN n | Time: 0.01 seconds | Results: 0


Nodes in the database:
